In [1]:
from rapidfuzz import fuzz, process, utils
import pandas as pd

In [2]:
dataset = pd.read_csv('../data/CleanData.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7136 entries, 0 to 7135
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product Name             7136 non-null   object 
 1   Selling Price($)         7136 non-null   float64
 2   About Product            7136 non-null   object 
 3   Product Specification    7136 non-null   object 
 4   Shipping Weight(Pounds)  7136 non-null   float64
 5   Main Category            7136 non-null   object 
 6   Sub-Category             7136 non-null   object 
 7   Side Category            6155 non-null   object 
dtypes: float64(2), object(6)
memory usage: 446.1+ KB


In [3]:
print(dataset['Product Name'][6305])

Mandala


In [4]:
products_list = dataset["Product Name"]
input = "Base Space"
limit = 5
ratio = process.extract(input, products_list, scorer=fuzz.ratio, limit=limit, processor=utils.default_process)
p_ratio = process.extract(input, products_list, scorer=fuzz.partial_ratio, limit=limit, processor=utils.default_process)
tsort_ratio = process.extract(input, products_list, scorer=fuzz.token_sort_ratio, limit=limit, processor=utils.default_process)
ptsort_ratio = process.extract(input, products_list, scorer=fuzz.partial_token_sort_ratio, limit=limit, processor=utils.default_process)
tset_ratio = process.extract(input, products_list, scorer=fuzz.token_set_ratio, limit=limit, processor=utils.default_process)
ptset_ratio = process.extract(input, products_list, scorer=fuzz.partial_token_set_ratio, limit=limit, processor=utils.default_process)
W_ratio = process.extract(input, products_list, scorer=fuzz.WRatio, limit=limit, processor=utils.default_process)
Q_ratio = process.extract(input, products_list, scorer=fuzz.QRatio, limit=limit, processor=utils.default_process)

print(f'ratio:                      {ratio}')
print(f'partial ratio:              {p_ratio[0]}')
print(f'                            {p_ratio[1]}')
print(f'token sort ratio:           {tsort_ratio}')
print(f'partial token ratio:        {ptsort_ratio}')
print(f'token set ratio:            {tset_ratio}')
print(f'partial token set ratio:    {ptset_ratio}')
print(f'Wratio:                     {W_ratio}')
print(f'Qratio:                     {Q_ratio}')

ratio:                      [('Space Base', 50.0, 4025), ('Barbie Car Wash Playset', 48.484848484848484, 1496), ("Rubie's Pet Costume", 48.275862068965516, 5808), ('Pan Balance', 47.61904761904761, 5031), ('Secrets', 47.05882352941176, 3844)]
partial ratio:              ("RoyalBaby Space Shuttle Kids Bike for Boys and Girls, 14 16 18 Inch Magnesium Bicycle with 2 Hand Disc Brakes, Child's Cycle with Training Wheels or Kickstand, Black White Purple", 80.0, 360)
                            ('Chic Home Spaceship 5 Piece Comforter Set Space Explorer Extraterrestrial Theme Youth Design Bedding-Throw Blanket Decorative Pillow Shams Included, Full', 80.0, 6621)
token sort ratio:           [('Space Base', 100.0, 4025), ('Clank!, Base', 60.0, 582), ('Specter Ops', 57.14285714285714, 3878), ('Pan Balance', 57.14285714285714, 5031), ('Lego Space & Airport Set', 56.25, 3461)]
partial token ratio:        [('Space Base', 100.0, 4025), ('Lego Space & Airport Set', 84.21052631578947, 3461), ('Pointeha

In [5]:
def prototype_rapid_fuzz_filter(user_input, products, number_of_rec):
    list_of_rec = []
    token_set_ratio_match = process.extract(user_input, products, scorer=fuzz.token_set_ratio, limit=1, processor=utils.default_process)
    partial_ratio_matches = process.extract(user_input, products, scorer=fuzz.token_set_ratio, limit=number_of_rec, processor=utils.default_process)

    if token_set_ratio_match[0][1] == 100:
        list_of_rec.extend(token_set_ratio_match)
        for match in partial_ratio_matches:
            if match[0] != token_set_ratio_match[0][0]:
                list_of_rec.append(match)
    
    else:
        for match in partial_ratio_matches:
            list_of_rec.append(match)

    return list_of_rec

def rapid_fuzz_rec_to_df(recommendations, dataframe):
    sorted_indeces = [match[2] for match in recommendations]
    reordered_df = dataframe.iloc[sorted_indeces].reset_index(drop=True)
    return reordered_df

In [6]:
print(dataset['Product Name'][2489])

Darice, 600 Mermaid Princess Sticker Book


In [7]:
input = 'Games'
list_recs = []
limit = 5
for i in range(limit):
    print(prototype_rapid_fuzz_filter(user_input=input, products=products_list, number_of_rec=limit)[i])
list_recs = prototype_rapid_fuzz_filter(user_input=input, products=products_list, number_of_rec=len(dataset['Product Name']))
new_df = rapid_fuzz_rec_to_df(recommendations=list_recs, dataframe=dataset)

print(new_df)

('GreenBrier Games Yashima Legends from Fairytale Board Game', 100.0, 193)
('University Games Break Out Game, Kids', 100.0, 217)
('Carson-Dellosa Publishing Language Arts Learning Games, Grade 2', 100.0, 250)
('Buffalo Games - New York Twilight - 300 Large Piece Jigsaw Puzzle', 100.0, 296)
('Loftus SW-0249 4 Pc Classic Wooden Games in A Tin Set, 6 inches Long, Brown', 100.0, 314)
                                           Product Name  Selling Price($)  \
0     GreenBrier Games Yashima Legends from Fairytal...             35.00   
1                 University Games Break Out Game, Kids             11.12   
2     Carson-Dellosa Publishing Language Arts Learni...             24.99   
3     Buffalo Games - New York Twilight - 300 Large ...             13.77   
4     Loftus SW-0249 4 Pc Classic Wooden Games in A ...             12.54   
...                                                 ...               ...   
7131  Tektrum 3 Piece 3pc Duvet Cover Set with Butto...             22.82   
7